In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from statsmodels.tsa.arima_model import ARIMA
from arch import arch_model
import seaborn as sns
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

C:\Users\Sruja\AppData\Local\Temp\ipykernel_13264\3031487806.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
raw_data = yfinance.download (tickers = "^GSPC ^FTSE ^N225 ^GDAXI", start = "1994-01-07", end = "2018-01-29", 
                              interval = "1d", group_by = 'ticker', auto_adjust = True)

[*********************100%%**********************]  4 of 4 completed


In [3]:
df_comp = raw_data.copy()

In [4]:
df_comp.head()

Ticker             ^N225                                                   \
Price               Open          High           Low         Close Volume   
Date                                                                        
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766    0.0   
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453    0.0   
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000    0.0   
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859    0.0   
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766    0.0   

Ticker            ^FTSE                                                \
Price              Open         High          Low        Close Volume   
Date                                                                    
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0   

Ticker           ^GSPC                                                   \
Price             Open        High         Low       Close       Volume   
Date                                                                      
1994-01-07  467.089996  470.260010  467.029999  469.899994  324920000.0   
1994-01-10  469.899994  475.269989  469.549988  475.269989  319490000.0   
1994-01-11  475.269989  475.279999  473.269989  474.130005  305490000.0   
1994-01-12  474.130005  475.059998  472.140015  474.170013  310690000.0   
1994-01-13  474.170013  474.170013  471.799988  472.470001  277970000.0   

Ticker           ^GDAXI                                                
Price              Open         High          Low        Close Volume  
Date                                                                   
1994-01-07  2218.959961  2227.639893  2201.820068  2224.949951    0.0  
1994-01-10  2231.840088  2238.010010  2222.000000  2225.000000    0.0  
1994-01-11  2225.429932  2235.610107  2225.179932  2228.100098    0.0  
1994-01-12  2227.120117  2227.790039  2182.060059  2182.060059    0.0  
1994-01-13  2171.500000  2183.709961  2134.100098  2142.370117    0.0

In [5]:
df_comp['spx'] = df_comp['^GSPC'].Close[:]
df_comp['dax'] = df_comp['^GDAXI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]

In [6]:
df_comp.head()

Ticker             ^N225                                                   \
Price               Open          High           Low         Close Volume   
Date                                                                        
1994-01-07  17842.980469  18131.410156  17787.480469  18124.009766    0.0   
1994-01-10  18186.519531  18567.060547  18186.519531  18443.439453    0.0   
1994-01-11  18481.849609  18671.669922  18373.039062  18485.250000    0.0   
1994-01-12  18447.339844  18807.080078  18301.929688  18793.880859    0.0   
1994-01-13  18770.380859  18823.380859  18548.750000  18577.259766    0.0   

Ticker            ^FTSE                                                ...  \
Price              Open         High          Low        Close Volume  ...   
Date                                                                   ...   
1994-01-07  3401.399902  3446.800049  3398.699951  3446.000000    0.0  ...   
1994-01-10  3465.699951  3468.100098  3430.000000  3440.600098    0.0  ...   
1994-01-11  3442.500000  3442.500000  3413.500000  3413.800049    0.0  ...   
1994-01-12  3394.800049  3402.399902  3372.000000  3372.000000    0.0  ...   
1994-01-13  3380.699951  3383.300049  3356.899902  3360.000000    0.0  ...   

Ticker            ^GSPC       ^GDAXI                                         \
Price            Volume         Open         High          Low        Close   
Date                                                                          
1994-01-07  324920000.0  2218.959961  2227.639893  2201.820068  2224.949951   
1994-01-10  319490000.0  2231.840088  2238.010010  2222.000000  2225.000000   
1994-01-11  305490000.0  2225.429932  2235.610107  2225.179932  2228.100098   
1994-01-12  310690000.0  2227.120117  2227.790039  2182.060059  2182.060059   
1994-01-13  277970000.0  2171.500000  2183.709961  2134.100098  2142.370117   

Ticker                    spx          dax         ftse        nikkei  
Price      Volume                                                      
Date                                                                   
1994-01-07    0.0  469.899994  2224.949951  3446.000000  18124.009766  
1994-01-10    0.0  475.269989  2225.000000  3440.600098  18443.439453  
1994-01-11    0.0  474.130005  2228.100098  3413.800049  18485.250000  
1994-01-12    0.0  474.170013  2182.060059  3372.000000  18793.880859  
1994-01-13    0.0  472.470001  2142.370117  3360.000000  18577.259766  

[5 rows x 24 columns]

In [7]:
df_comp = df_comp.iloc[1:]
del df_comp['^N225']
del df_comp['^GSPC']
del df_comp['^GDAXI']
del df_comp['^FTSE']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

## creating Returns

In [8]:
df_comp['ret_spx'] = df_comp.spx.pct_change(1)*100
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1)*100
df_comp['ret_dax'] = df_comp.dax.pct_change(1)*100
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1)*100

## splitting the data

In [9]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

## Fitting a Model

In [10]:
from pmdarima.arima import auto_arima

In [11]:
model_auto = auto_arima(df.ret_ftse[1:])

In [12]:
model_auto

ARIMA(order=(4, 0, 5), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [13]:
model_auto.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 5019
Model:               SARIMAX(4, 0, 5)   Log Likelihood               -7882.776
Date:                Sun, 03 Mar 2024   AIC                          15785.552
Time:                        23:32:05   BIC                          15850.762
Sample:                    01-11-1994   HQIC                         15808.403
                         - 04-05-2013                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0121      0.082      0.148      0.883      -0.148       0.172
ar.L2         -0.6542      0.077     -8.456      0.000      -0.806      -0.503
ar.L3         -0.1628      0.071     -2.290      0.022      -0.302      -0.023
ar.L4          0.2014      0.074      2.712      0.007       0.056       0.347
ma.L1         -0.0357      0.081     -0.440      0.660      -0.195       0.123
ma.L2          0.6067      0.078      7.768      0.000       0.454       0.760
ma.L3          0.0621      0.068      0.908      0.364      -0.072       0.196
ma.L4         -0.1934      0.073     -2.650      0.008      -0.336      -0.050
ma.L5         -0.1052      0.010    -11.067      0.000      -0.124      -0.087
sigma2         1.3543      0.015     91.439      0.000       1.325       1.383
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              6354.65
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               1.99   Skew:                            -0.20
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.50
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [18]:
model_auto_1 = auto_arima(df_comp.ret_ftse[1:], exogenous = df_comp[['ret_spx', 'ret_dax', 'ret_nikkei']][1:], m = 5,
                       max_order = None, max_p = 7, max_q = 7, max_d = 2, max_P = 4, max_Q = 4, max_D = 2,
                       maxiter = 50, alpha = 0.05, n_jobs = -1, trend = 'ct', information_criterion = 'oob',
                       out_of_sample = int(len(df_comp)*0.2))


# !!! Important Note: In pdmarima v1.5.2, out_of_sample_size is replaced with out_of_sample, so make sure to use the latter!


# exogenous -> outside factors (e.g other time series)
# m -> seasonal cycle length
# max_order -> maximum amount of variables to be used in the regression (p + q)
# max_p -> maximum AR components
# max_q -> maximum MA components
# max_d -> maximum Integrations
# maxiter -> maximum iterations we're giving the model to converge the coefficients (becomes harder as the order increases)
# alpha -> level of significance, default is 5%, which we should be using most of the time
# n_jobs -> how many models to fit at a time (-1 indicates "as many as possible")
# trend -> "ct" usually
# information_criterion -> 'aic', 'aicc', 'bic', 'hqic', 'oob' 
#        (Akaike Information Criterion, Corrected Akaike Information Criterion,
#        Bayesian Information Criterion, Hannan-Quinn Information Criterion, or
#        "out of bag"--for validation scoring--respectively)
# out_of_smaple -> validates the model selection (pass the entire dataset, and set 20% to be the out_of_sample_size)

In [19]:
model_auto_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 6274
Model:             SARIMAX(0, 0, 4)x(0, 0, [1], 5)   Log Likelihood               -9560.337
Date:                             Mon, 04 Mar 2024   AIC                          19136.673
Time:                                     00:30:27   BIC                          19190.627
Sample:                                 01-11-1994   HQIC                         19155.368
                                      - 01-26-2018                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0246      0.028      0.884      0.377      -0.030       0.079
drift      -2.025e-06   7.91e-06     -0.256      0.798   -1.75e-05    1.35e-05
ma.L1         -0.0201      0.008     -2.496      0.013      -0.036      -0.004
ma.L2         -0.0496      0.007     -6.925      0.000      -0.064      -0.036
ma.L3         -0.0694      0.007     -9.724      0.000      -0.083      -0.055
ma.L4          0.0333      0.008      4.385      0.000       0.018       0.048
ma.S.L5       -0.0513      0.007     -7.261      0.000      -0.065      -0.037
sigma2         1.2386      0.012    104.270      0.000       1.215       1.262
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):              8783.36
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.86   Skew:                            -0.17
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.79
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""